In [12]:
import clip
import torch

from PIL import Image



In [17]:
image_path = 'E:/LAB/Grounding/CLIP/data/train/dog.0.jpg'
image = Image.open(image_path)
text = ['photo of a cat', 'photo of a dog']
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = clip.tokenize(text).to(device)
with torch.no_grad():
    logits_per_image, logits_per_text = model(image_input, text_inputs)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)

Label probs: [[0.00522 0.9946 ]]


In [15]:
model.train()
for p in model.parameters():
    if p.requires_grad == False:
        print(p)

In [9]:
model(image_input, text_inputs)

(tensor([[24.3594, 25.6250]], device='cuda:0', dtype=torch.float16,
        grad_fn=<MmBackward0>),
 tensor([[24.3594],
         [25.6250]], device='cuda:0', dtype=torch.float16, grad_fn=<TBackward0>))

In [7]:
logits_per_text, logits_per_image

(tensor([[24.3594],
         [25.6250]], device='cuda:0', dtype=torch.float16),
 tensor([[24.3594, 25.6250]], device='cuda:0', dtype=torch.float16))

In [1]:
import pandas as pd

test = pd.read_csv('E:/LAB/Grounding/CLIP/data/test.csv')
good = test[test['label'] == 'good']
bad = test[test['label'] == 'bad']

In [2]:
len(good), len(bad)

(71, 71)

In [4]:
import os
import shutil
from tqdm import tqdm

for i in tqdm(range(len(good))):
    shutil.copy(os.path.join('E:/LAB/Grounding/CLIP/data/all_img', os.path.basename(good.iloc[i]['image_path'])), 'E:/LAB/Grounding/CLIP/data/good_img/')
for i in tqdm(range(len(bad))):
    shutil.copy(os.path.join('E:/LAB/Grounding/CLIP/data/all_img', os.path.basename(bad.iloc[i]['image_path'])), 'E:/LAB/Grounding/CLIP/data/bad_img/')

100%|██████████| 71/71 [00:00<00:00, 631.99it/s]


In [1]:
from glob import glob
cats = glob('E:/LAB/Grounding/CLIP/data/train/cat*')
dogs = glob('E:/LAB/Grounding/CLIP/data/train/dog*')

In [2]:
len(cats), len(dogs)

(12500, 12500)

In [18]:
import random
train_test_cats = random.sample(cats, 271)
train_test_dogs = random.sample(dogs, 271)
train_cats = train_test_cats[:200]
test_cats = train_test_cats[200:]
train_dogs = train_test_dogs[:200]
test_dogs = train_test_dogs[200:]
train = train_cats + train_dogs
test = test_cats + test_dogs
train_label = ['cat'] * 200 + ['dog'] * 200
test_label = ['cat'] * 71 + ['dog'] * 71

In [5]:
def build_csv(imgs, label, save_path):
    df = pd.DataFrame()
    df['image_path'] = imgs
    df['label'] = label
    df.to_csv(save_path, index=False)

In [19]:
import pandas as pd
build_csv(train, train_label, 'E:/LAB/Grounding/CLIP/data/catsdogs_train.csv')

In [20]:
build_csv(test, test_label, 'E:/LAB/Grounding/CLIP/data/catsdogs_test.csv')

In [1]:
label = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
predication = [0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1]

In [2]:
wrongcase = []
for i in range(len(label)):
    if label[i] != predication[i]:
        wrongcase.append(i)

len(wrongcase)

51

In [6]:
import pandas as pd
test = pd.read_csv('./data/test.csv')

In [7]:
test.iloc[0]

image_path    /home/zhaoyue/C4Web_final/newscreenshots_bbox/...
label                                                       bad
Name: 0, dtype: object

In [8]:
len(test)

142

In [14]:
fail_path = test['image_path'][wrongcase].tolist()
fail_label = test['label'][wrongcase].tolist()
import shutil
import os
for i in range(len(fail_path)):
    shutil.copy(os.path.join('./data/all_img/', os.path.basename(fail_path[i])), f'./data/clip_fail/{os.path.basename(fail_path[i]).split(".")[0] + fail_label[i] + "." + os.path.basename(fail_path[i]).split(".")[1]}')

In [2]:
import pandas as pd
train = pd.read_csv('./data/train_aug.csv')